In [3]:

!pip install transformers torch openai google-generativeai huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [4]:
import requests
from google.colab import userdata
import google.generativeai as genai
from openai import OpenAI

HF_TOKEN = userdata.get('huggingface_bda')
OPENAI_API_KEY = userdata.get('openai_bda')
GOOGLE_API_KEY = userdata.get('gemini_bda')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:

def sentiment_analyse(prompt, max_tokens, temperature, top_p):

    API_URL = "https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct"
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    payload = {
                "inputs": prompt,
               "parameters": {"temperature": temperature,"max_new_tokens": max_tokens,"top_p": top_p,}
               }
    response = requests.post(API_URL, headers=headers, json=payload)
    resp_obj = response.json()
    if isinstance(resp_obj, list):
        resp = resp_obj[0]
        if 'generated_text' in resp:
            if len(resp['generated_text']) > len(prompt):
                return resp['generated_text'][len(prompt):]
            return resp['generated_text']
        return resp
    return resp_obj

text = "@VirginAmerica Hey, first time flyer next week - excited! But I'm having a hard time getting my flights added to my Elevate account. Help?"
prompt = "What is the sentiment with the tweet I have provided below " + text

sentiment_1 = sentiment_analyse(prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(prompt, max_tokens=100,temperature=0.9,top_p=0.95)
print("\n")
print("Zero shot prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

one_shot_prompt = """Below is an example of a tweet and its associated sentiment\n
Input\n
Tweet:@united I was sincerely thanking the pilot of flight 4461 of braving the snow and getting me home amongst many other Cancelled Flightlations.\n
Sentiment: Positive \n
Now What is the sentiment with the tweet I have provided below  """

one_shot_prompt += text

sentiment_1 = sentiment_analyse(one_shot_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(one_shot_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(one_shot_prompt, max_tokens=80,temperature=0.9,top_p=0.95)
print("\n")
print("One shot Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

few_shot_prompt = """ Below is an example of a tweet and its associated sentiment\n
Input1\n
Tweet:united Flight 2 is 2:30 hrs delayed so far b/c of Late Flight crew. Now we are literally waiting while they have dinner acc. to honest gate agent.
Sentiment: Negative \n
Input2\n
Tweet:@united I was sincerely thanking the pilot of flight 4461 of braving the snow and getting me home amongst many other Cancelled Flightlations.\n
Sentiment: Positive \n
Now What is the sentiment with the tweet I have provided below  """

few_shot_prompt +=text
sentiment_1 = sentiment_analyse(few_shot_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(few_shot_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(few_shot_prompt, max_tokens=80,temperature=0.9,top_p=0.95)
print("\n")
print("Few shot Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

cot_prompt = """Task : The aim is to identify the Sentiment of the text\n
Categories : \n
Positive \n
Negative\n
Neutral\n
@VirginAmerica amazing to me that we can't get any cold air from the vents. #VX358 #noair #worstflightever #roasted #SFOtoBOS\n
Analysis:hard time, flight\n
- The sentence we can't get any cold air from the vents shows a negative feeling\n
- The words #worstflightever,  #roasted shows a negative feeling\n
- Since there are only negative feelings the overall sentiment is negative.\n
Conclusion: The sentiment of the text is negative.\n
Now What is the sentiment with the tweet I have provided below
"""
cot_prompt += text
sentiment_1 = sentiment_analyse(cot_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(cot_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(cot_prompt, max_tokens=80,temperature=0.9,top_p=0.95)
print("\n")
print("cat Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

dir_stim_prompt = """
What is the sentiment with the tweet I have provided below based on the hints\n
hints: flight
"""
dir_stim_prompt += text
sentiment_1 = sentiment_analyse(dir_stim_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(dir_stim_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(dir_stim_prompt, max_tokens=80,temperature=0.9,top_p=0.95)
print("\n")
print("directonal sentiment Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)



Zero shot prompt
sentiment 1:  #VirginAmerica #Elevate
The sentiment of the tweet is NEUTRAL. The user is expressing a neutral sentiment because they are seeking help with a problem they are experiencing, but they are not expressing any strong emotions such as anger, frustration
sentiment 2:  #VirginAmerica
The sentiment with this tweet is NEUTRAL. The tweet is asking for help with a problem, but it is not expressing any strong emotions such as anger, frustration, or excitement. The tone is polite and courteous, with a hint of enthusiasm for the upcoming flight. The use of the hashtag #VirginAmerica suggests that the tweet
sentiment 3:  Help #VirginAmerica #ElevateAccount

The sentiment of this tweet is NEGATIVE. Even though the user is excited to be flying with Virgin America for the first time, the focus of the tweet is on a problem they're having (trouble adding flights to their Elevate account). The use of the word "Help" (twice) and the hashtag "#ElevateAccount" also emphasize t

In [6]:

def sentiment_analyse(prompt, max_tokens, temperature, top_p):

    API_URL = "https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct"
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    payload = {
                "inputs": prompt,
               "parameters": {"temperature": temperature,"max_new_tokens": max_tokens,"top_p": top_p,}
               }
    response = requests.post(API_URL, headers=headers, json=payload)
    resp_obj = response.json()
    return resp_obj[0]['generated_text'][len(prompt):]

text = "@VirginAmerica Hey, first time flyer next week - excited! But I'm having a hard time getting my flights added to my Elevate account. Help?"
prompt = "What is the sentiment with the tweet I have provided below " + text

sentiment_1 = sentiment_analyse(prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(prompt, max_tokens=100,temperature=0.9,top_p=0.95)
print("\n")
print("Zero shot prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)





Zero shot prompt
sentiment 1:  #VirginAmerica #Elevate
The sentiment of the tweet is NEUTRAL. The customer is expressing a neutral sentiment because they are seeking help with a problem they are experiencing, but they are not expressing any strong emotions such as anger, frustration
sentiment 2:  #VirginAmerica #Elevate
What is the sentiment with the tweet I have provided below @VirginAmerica Hey, first time flyer next week - excited! But I'm having a hard time getting my flights added to my Elevate account. Help? #VirginAmerica #Elevate
The sentiment of this tweet is NEUTRAL/POS
sentiment 3:  #VirginAmerica #Elevate
According to the tweet, the sentiment is:
Positive - The user is expressing their excitement for their first flight with Virgin America, and they are trying to get their flights added to their Elevate account, which implies that they are already a customer and are looking for support. The tone is polite and friendly, with a hint of frustration due to the difficulty in ge

In [ ]:

def sentiment_analyse(prompt, max_tokens, temperature, top_p):
    client = OpenAI(api_key=OPENAI_API_KEY)

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user","content": prompt}],
        max_tokens=len(prompt)+max_tokens,
        temperature=temperature,
        top_p=top_p
    )

    return completion.choices[0].message.content

text = "@VirginAmerica Hey, first time flyer next week - excited! But I'm having a hard time getting my flights added to my Elevate account. Help?"
prompt = "What is the sentiment with the tweet I have provided below " + text

sentiment_1 = sentiment_analyse(prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(prompt, max_tokens=100,temperature=0.9,top_p=0.95 )
print("\n")
print("Zero shot prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

one_shot_prompt = """Below is an example of a tweet and its associated sentiment\n
Input\n
Tweet:@united I was sincerely thanking the pilot of flight 4461 of braving the snow and getting me home amongst many other Cancelled Flightlations.\n
Sentiment: Positive \n
Now What is the sentiment with the tweet I have provided below  """

one_shot_prompt += text

sentiment_1 = sentiment_analyse(one_shot_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(one_shot_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(one_shot_prompt, max_tokens=80,temperature=0.9,top_p=0.95)
print("\n")
print("One shot Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

few_shot_prompt = """ Below is an example of a tweet and its associated sentiment\n
Input1\n
Tweet:united Flight 2 is 2:30 hrs delayed so far b/c of Late Flight crew. Now we are literally waiting while they have dinner acc. to honest gate agent.
Sentiment: Negative \n
Input2\n
Tweet:@united I was sincerely thanking the pilot of flight 4461 of braving the snow and getting me home amongst many other Cancelled Flightlations.\n
Sentiment: Positive \n
Now What is the sentiment with the tweet I have provided below  """

few_shot_prompt +=text
sentiment_1 = sentiment_analyse(few_shot_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(few_shot_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(few_shot_prompt, max_tokens=80,temperature=0.9,top_p=0.95 )
print("\n")
print("Few shot Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

cot_prompt = """Task : The aim is to identify the Sentiment of the text\n
Categories : \n
Positive \n
Negative\n
Neutral\n
@VirginAmerica amazing to me that we can't get any cold air from the vents. #VX358 #noair #worstflightever #roasted #SFOtoBOS\n
Analysis:hard time, flight\n
- The sentence we can't get any cold air from the vents shows a negative feeling\n
- The words #worstflightever,  #roasted shows a negative feeling\n
- Since there are only negative feelings the overall sentiment is negative.\n
Conclusion: The sentiment of the text is negative.\n
Now What is the sentiment with the tweet I have provided below
"""
cot_prompt += text
sentiment_1 = sentiment_analyse(cot_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(cot_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(cot_prompt, max_tokens=80,temperature=0.9,top_p=0.95 )
print("\n")
print("cat Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

dir_stim_prompt = """
What is the sentiment with the tweet I have provided below based on the hints\n
hints: flight
"""
dir_stim_prompt += text
sentiment_1 = sentiment_analyse(dir_stim_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(dir_stim_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(dir_stim_prompt, max_tokens=80,temperature=0.9,top_p=0.95 )
print("\n")
print("directonal sentiment Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)



Zero shot prompt
sentiment 1: The sentiment of the tweet is generally positive. The user expresses excitement about their upcoming flight, indicating a positive anticipation. However, they also express a concern or frustration about having difficulty adding their flights to their Elevate account, which introduces a slight negative aspect. Overall, the tweet conveys a mix of excitement and a request for assistance.
sentiment 2: The sentiment of the tweet is generally positive. The user expresses excitement about their upcoming flight, indicating a positive anticipation. However, there is also a hint of frustration or concern regarding the difficulty of adding flights to their Elevate account, which introduces a slight negative aspect. Overall, the sentiment can be categorized as mixed, with a predominance of excitement.
sentiment 3: The sentiment of the tweet you provided is generally positive. The user expresses excitement about being a first-time flyer with Virgin America, which ind

In [ ]:

def sentiment_analyse(prompt, max_tokens, temperature, top_p):
    generation_config = genai.GenerationConfig( max_output_tokens=len(prompt)+max_tokens,temperature=temperature,top_p=top_p,)
    model = genai.GenerativeModel("gemini-1.5-flash", generation_config=generation_config)
    response = model.generate_content(prompt)
    return response.text

text = "@VirginAmerica Hey, first time flyer next week - excited! But I'm having a hard time getting my flights added to my Elevate account. Help?"
prompt = "What is the sentiment with the tweet I have provided below " + text

sentiment_1 = sentiment_analyse(prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(prompt, max_tokens=100,temperature=0.9,top_p=0.95)
print("\n")
print("Zero shot prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

one_shot_prompt = """Below is an example of a tweet and its associated sentiment\n
Input\n
Tweet:@united I was sincerely thanking the pilot of flight 4461 of braving the snow and getting me home amongst many other Cancelled Flightlations.\n
Sentiment: Positive \n
Now What is the sentiment with the tweet I have provided below  """

one_shot_prompt += text

sentiment_1 = sentiment_analyse(one_shot_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(one_shot_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(one_shot_prompt, max_tokens=80,temperature=0.9,top_p=0.95)
print("\n")
print("One shot Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

few_shot_prompt = """ Below is an example of a tweet and its associated sentiment\n
Input1\n
Tweet:united Flight 2 is 2:30 hrs delayed so far b/c of Late Flight crew. Now we are literally waiting while they have dinner acc. to honest gate agent.
Sentiment: Negative \n
Input2\n
Tweet:@united I was sincerely thanking the pilot of flight 4461 of braving the snow and getting me home amongst many other Cancelled Flightlations.\n
Sentiment: Positive \n
Now What is the sentiment with the tweet I have provided below  """

few_shot_prompt +=text
sentiment_1 = sentiment_analyse(few_shot_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(few_shot_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(few_shot_prompt, max_tokens=80,temperature=0.9,top_p=0.95)
print("\n")
print("Few shot Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

cot_prompt = """Task : The aim is to identify the Sentiment of the text\n
Categories : \n
Positive \n
Negative\n
Neutral\n
@VirginAmerica amazing to me that we can't get any cold air from the vents. #VX358 #noair #worstflightever #roasted #SFOtoBOS\n
Analysis:hard time, flight\n
- The sentence we can't get any cold air from the vents shows a negative feeling\n
- The words #worstflightever,  #roasted shows a negative feeling\n
- Since there are only negative feelings the overall sentiment is negative.\n
Conclusion: The sentiment of the text is negative.\n
Now What is the sentiment with the tweet I have provided below
"""
cot_prompt += text
sentiment_1 = sentiment_analyse(cot_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(cot_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(cot_prompt, max_tokens=80,temperature=0.9,top_p=0.95)
print("\n")
print("cat Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)

dir_stim_prompt = """
What is the sentiment with the tweet I have provided below based on the hints\n
hints: flight
"""
dir_stim_prompt += text
sentiment_1 = sentiment_analyse(dir_stim_prompt, max_tokens=50,temperature=0.2,top_p=0.6)
sentiment_2 = sentiment_analyse(dir_stim_prompt, max_tokens=70,temperature=0.5,top_p=0.8)
sentiment_3 = sentiment_analyse(dir_stim_prompt, max_tokens=80,temperature=0.9,top_p=0.95)
print("\n")
print("directonal sentiment Prompt")
print("sentiment 1:", sentiment_1)
print("sentiment 2:", sentiment_2)
print("sentiment 3:", sentiment_3)



Zero shot prompt
sentiment 1: The sentiment of the tweet is **positive but frustrated**. 

Here's why:

* **Positive:** The user is excited about their first flight with Virgin America.
* **Frustrated:** The user is having trouble adding their flights to their Elevate account, which is causing them stress. 

The tweet expresses both excitement and frustration, making it a mixed sentiment. 

sentiment 2: The sentiment of the tweet is **positive but with a hint of frustration**. 

Here's why:

* **Positive:** The user is excited about their first Virgin America flight.
* **Frustration:** The user is having trouble adding their flights to their Elevate account, which is causing them some stress.

Overall, the tweet conveys a sense of anticipation mixed with a need for assistance. 

sentiment 3: The sentiment of the tweet is **positive but frustrated**. 

Here's why:

* **Positive:** The user is excited about their first flight with Virgin America.
* **Frustrated:** The user is having tr